1. Get initial set of features.
2.  Get those with low utility. (Note: all candidates have some utility, as they are post filt)
3.  remove those found in novel candidates (after varied filterings)
4.  Get KG distance - keep cases distance 2 or (or less?)
5.  OPT: do literature search to see if results found? (maybe only enough for ranking ) - e.g. has any res
6.  Opt:Filter with simpler LLM to get those marked as non novel, non plausible.
* Opt: +- medRag (speed)? 
6.  Invent fake, positive explanation for utility, plausability, "interesting". (maybe in 1 call?)

Finally, we'll add them to candidates for annotation


In [1]:
#!/usr/bin/env python
# coding: utf-8
import pandas as pd
import numpy as np
from IPython import get_ipython
from IPython.core.display_functions import display
from tqdm import tqdm  # Import tqdm for the progress bar

## scispacy, medspacy, medcat, quickumls, semrep...
import networkx as nx
from itertools import combinations

import spacy
import scispacy
from itertools import compress
from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector ## https://github.com/allenai/scispacy?tab=readme-ov-file#example-usage

import sys, os ## append parent path to dir to allow import
parent_directory = os.path.abspath('..')
sys.path.append(parent_directory)
# from util import get_sentence_pairs_similarity, wrangle_df_icu, anti_join_df,test_nel,get_multiple_nel,link_kg_concepts_slim, generate_boring_prompts
from util import * # deduplicate texts

import string
rem = string.punctuation
punct_pattern = r"[{}]".format(rem)
# get_ipython().run_line_magic('load_ext', 'autoreload')
# get_ipython().run_line_magic('autoreload', '2')
from configs import * #config_gall
import logging
import warnings
# warnings.filterwarnings('ignore', message=".*UserWarning*.")
# warnings.filterwarnings('ignore', message='.*Duplicate section*.')

# Suppress specific warnings
warnings.simplefilter("ignore", UserWarning)  # Catch UserWarnings more broadly
warnings.filterwarnings("ignore", message=".*The component 'matcher' does not have any patterns defined.*")

# Set up logger to suppress spaCy's warnings
logger = logging.getLogger("spacy")
logger.setLevel(logging.ERROR)

from Link_semmed_cuis import  * #link_kg_concepts #*
from ukb_target_pipe  import  make_target_df,model_features,ipw_downsampling#*
from search_pubmed import  *# run_search_pubmed #*
from configs import * #config_gall

import nltk
lemma = nltk.wordnet.WordNetLemmatizer()

/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# FastRun = True
FastRun = False
Save_Outputs= True
generate_basic_neg_candidates = True #False
OutputPath = "./Outputs/Negatives/"

In [4]:
from configs import *#  config_celiac,config_gall,  config_gout

## import from configs

if FastRun:
    all_configs = [config_celiac] # config_gall , config_eye_occ
else:
    all_configs = [
    config_gall,  config_eye_occ,
      config_gout,config_celiac, config_spine, config_oesophagus, config_heart, config_depression
              ] 
config = all_configs[0]

config

{'TARGET_CODES_LIST': ('K80', 'K81', 'K82'),
 'do_IPW': True,
 'do_boruta_fs': True,
 'K_IPW_RATIO': 9,
 'targets': ['Cholelithiasis',
  'Gallstone',
  'Gallbladder disease',
  'cholecystitis',
  'Cholangitis'],
 'FEATURES_REPORT_PATH': 'gallstone_ipw_broad_feature_report.csv',
 'QUERY_CANDIDATES_FILE': 'candidate_novel_cuis_chol.csv',
 'CANDIDATE_NOVEL_CUIS_FILEPATH': 'candidate_novel_cuis_chol.csv',
 'DO_MINI_COMBINED_PATH_FILT': False,
 'SAVE_OUTPUTS': True,
 'TARGET_NAME': 'GALLSTONES, Cholelithiasis',
 'additional_target_cui_terms_list': ['C0008325', 'C0008311'],
 'OUTPUT_RES_PREFIX': 'gallstone_'}

In [5]:
if generate_basic_neg_candidates:
    if FastRun:
        nlp = spacy.load("en_core_sci_sm")
    else:
        nlp = spacy.load("en_core_sci_lg")
    # nlp = spacy.load("en_core_sci_scibert")
    nlp.add_pipe("abbreviation_detector")
    nlp.add_pipe("scispacy_linker",
                 config={"resolve_abbreviations": True,"linker_name": "umls"
                         ,"max_entities_per_mention": 2  # 3,
                     ,"threshold": 0.85  ## default is 0.8, paper mentions 0.99 as thresh
                         })


/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [6]:
def mini_pubmed_search(df):
    Entrez.email = config.get('ENTREZ_EMAIL', os.environ.get('ENTREZ_EMAIL'))
    Entrez.api_key = config.get('ENTREZ_API_KEY', os.environ.get('ENTREZ_API_KEY'))
    init_db()
    
    # Process targets
    targets = config['targets']
    if len(targets) > 1:
        targets = [" OR ".join("({})".format(x) for x in targets)]
    print("Targets:", targets)
    
    # # Load candidate queries
    # if df_query is None:
    #     df_query = load_candidate_queries(config['QUERY_CANDIDATES_FILE'], do_path_filter=config.get('DO_MINI_COMBINED_PATH_FILT', False))
    df_query = df.sort_values(["shortest_path_length","p_val","KG_Hits"],ascending=False).drop_duplicates("feature_name").reset_index(drop=True)
    
    # # Add feature names to queries
    # df_query = add_fnames_to_queries(df_query)
    print(f"{df_query.shape[0]} Rows, {df_query['feature_name'].nunique()} # unique Feature Names")
    
    # Get feature name queries
    fname_queries_subset = set(df_query["feature_name"].str.strip().unique())
    # Collect search data for feature names and targets
    fname_result_df = collect_search_data(fname_queries_subset, targets).drop_duplicates()
    print(fname_result_df.shape[0])
    ## note - some queries nigh identical, but it's more due to raw_name. So - it's ok... 
    fname_result_df = fname_result_df.loc[fname_result_df["Query"].str.lower().isin(df_sub["feature_name"].str.lower())]
    print(fname_result_df.shape[0])
    return fname_result_df


# import re
# from difflib import get_close_matches
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import AgglomerativeClustering

# def clean_text(text):
#     # Remove text inside parentheses along with the parentheses
#     cleaned = re.sub(r'\(.*?\)', '', text)
#     # Remove "None of the above" (case-insensitive)
#     cleaned = re.sub(r'(?i)None of the above', '', cleaned)
#     # Remove single-digit numbers as standalone words
#     cleaned = re.sub(r'\b\d\b', '', cleaned)
#     # Strip whitespace
#     cleaned = cleaned.strip()
#     return cleaned

# def deduplicate_texts(
#     texts, 
#     use_difflib=True, 
#     string_cutoff=0.95, 
#     distance_threshold=0.5
# ):
#     """
#     Deduplicate texts using a two-step process (optional difflib first, then semantic clustering),
#     while preserving the original texts. 
    
#     Steps:
#     1. Clean texts with regex-based rules for clustering only (originals are preserved).
#     2. (Optional) Use difflib-based merging on cleaned texts to quickly reduce near-duplicates.
#     3. Use semantic clustering on the reduced set to further deduplicate.
#     4. Return original texts as final output.

#     Parameters:
#         texts (list of str): The input texts.
#         use_difflib (bool): Whether to use difflib-based merging before semantic clustering.
#         string_cutoff (float): Cutoff for difflib.get_close_matches.
#         distance_threshold (float): Distance threshold for AgglomerativeClustering.
        
#     Returns:
#         list of str: Deduplicated texts (original, unmodified).
#     >>>
#         texts = [
#         'Ethnic background British', 'Ethnic background Mixed',
#         'Ethnic background',
#         'Doctor diagnosed COPD (chronic obstructive pulmonary disease)',
#         'Illnesses of mother 3 None (group 1)',
#         'Ethnic background White and Black Caribbean',
#         'Illnesses of siblings 1 Severe depression', 'osteoarthritis',
#         'Illnesses of siblings 1 Do not know (group 1)',
#         'Illnesses of siblings 2 Do not know (group 2)',
#         'Illnesses of mother 3 Do not know (group 2)',
#         'Ethnic background Indian', '(BD) bipolar disorder genetic risk',
#         'Ethnic background African', 'Ethnic background Chinese',
#         'Ever addicted to any substance or behaviour',
#         'Ever addicted to illicit or recreational drugs',
#         'Illnesses of siblings', 'Tobacco use disorder', 'sclerosis',
#         'Illnesses of siblings 2 Do not know (group 1)',
#         'Essential hypertension',
#         'Illnesses of siblings 1 Do not know (group 2)',
#         'Illnesses of siblings 3 Do not know (group 1)',
#         'Illnesses of siblings 3 Do not know (group 2)',
#     ]
#     final_texts = deduplicate_texts(texts, use_difflib=True, string_cutoff=0.95, distance_threshold=1.0)
#     """
#     print(len(texts), "# inputs")

#     # Pre-clean all texts (for clustering purposes only, no changes to originals)
#     cleaned_texts = [clean_text(t) for t in texts]

#     # Optional difflib-based merging step
#     if use_difflib:
#         print("# Applying string-level merging (difflib) first...")
#         reduced_originals = []
#         reduced_cleaned = []

#         for orig, ctext in zip(texts, cleaned_texts):
#             match = get_close_matches(ctext, reduced_cleaned, n=2, cutoff=string_cutoff)
#             if not match:
#                 # No match, so add this text
#                 reduced_originals.append(orig)
#                 reduced_cleaned.append(ctext)
#             else:
#                 # If there's a match, find its index
#                 existing_clean = match[0]
#                 idx = reduced_cleaned.index(existing_clean)
#                 existing_orig = reduced_originals[idx]

#                 # Pick the shortest original text
#                 if len(orig) < len(existing_orig):
#                     reduced_originals[idx] = orig
#                     reduced_cleaned[idx] = ctext
#         texts_for_semantic = reduced_originals
#     else:
#         texts_for_semantic = texts

#     # Re-clean for semantic step (in case we skipped difflib or changed sets)
#     cleaned_for_semantic = [clean_text(t) for t in texts_for_semantic]
    
#     model = SentenceTransformer("FremyCompany/BioLORD-2023") #'sentence-transformers/all-MiniLM-L12-v2'
#     embeddings = model.encode(cleaned_for_semantic, show_progress_bar=False)

#     # print("# Performing semantic clustering...")
#     clusterer = AgglomerativeClustering(
#         n_clusters=None, 
#         distance_threshold=distance_threshold, 
#         linkage='average', 
#         metric='euclidean'
#     )
#     labels = clusterer.fit_predict(embeddings)

#     # print("# Selecting shortest representative from each semantic cluster...")
#     cluster_map = {}
#     for label, orig_text in zip(labels, texts_for_semantic):
#         if label not in cluster_map or len(orig_text) < len(cluster_map[label]):
#             cluster_map[label] = orig_text

#     final = list(cluster_map.values())
#     print(len(final), "# Deduplicated.")

#     return final

#### our original utils filter didn't leave in enough "Weak" features  - rerun here
* I rerun it here after changing filt, and use intermediate df. ( I don't want to overwrite the existing stuff adn pay for reunning it all) . 

"""
for config in all_configs:
    try:
        print("Running pipeline for:", config["TARGET_NAME"])
        df = make_target_df(TARGET_CODES_LIST=config["TARGET_CODES_LIST"], FAST=FastRun)
        print("-------"*9, "\nTarget extracted: df.shape\n", "-------"*12)
        df.to_parquet(config["OUTPUT_RES_PREFIX"]+"_raw_df.parquet",index=False) # save output - not present in orig run
        
        # if config.get("do_IPW", True):
        df = ipw_downsampling(df, K_IPW_RATIO=3)
        print(df.shape, "IPW downsampled")
        
        ### not sure if to do this step:
        ## already extracted list of existing, likely novel cases - drop these out already to save calc time (note - will affect feature importance!!!)
        df_pos_candidates = pd.read_csv(config['CANDIDATE_NOVEL_CUIS_FILEPATH']) # cases filtered for novelty
        df.drop(columns=df_pos_candidates["raw_name"],errors="ignore",inplace=True)
        print(df.shape,"after dropping known good features")
        
        res_dict = model_features(df=df, FAST=FastRun,
                                  do_boruta_fs=False,do_stat_fs_filt=False,
                                  CORR_THRESH=0.85,
                                  SAVE_OUTPUT=False,
                                  # FEATURES_REPORT_PATH=config["FEATURES_REPORT_PATH"],
                                 return_res=True,
                                  pval_ceiling=0.999 # keep all features/weak feats
                                 # get_feature_importances=True ## if false, then return only pipeline, feature names + skip filtering features
                                 )

    except Exception as e:
        print(f"Error processing {config['TARGET_NAME']}: {e}")
"""

# df_feat = res_dict['feature_importance_df']
df_feat = pd.read_csv(config['FEATURES_REPORT_PATH']) # all candidate (minimal utiltity) features
df_pos_candidates = pd.read_csv(config['CANDIDATE_NOVEL_CUIS_FILEPATH']) # cases filtered for novelty
df_feat = df_feat.loc[~df_feat["raw_name"].isin(df_pos_candidates["raw_name"])]
### get low utility features:
# df_feat = df_feat.loc[(df_feat["feature_importance"]<0.001)& (df_feat["p_val"]>0.1)& (df_feat["MutualInfoTarget"]<0.005)]

df_feat.loc[(df_feat["feature_importance"]<=0.05)& (df_feat["p_val"]>0.07)].reset_index(drop=True).round(2)


In [7]:
config

{'TARGET_CODES_LIST': ('K80', 'K81', 'K82'),
 'do_IPW': True,
 'do_boruta_fs': True,
 'K_IPW_RATIO': 9,
 'targets': ['Cholelithiasis',
  'Gallstone',
  'Gallbladder disease',
  'cholecystitis',
  'Cholangitis'],
 'FEATURES_REPORT_PATH': 'gallstone_ipw_broad_feature_report.csv',
 'QUERY_CANDIDATES_FILE': 'candidate_novel_cuis_chol.csv',
 'CANDIDATE_NOVEL_CUIS_FILEPATH': 'candidate_novel_cuis_chol.csv',
 'DO_MINI_COMBINED_PATH_FILT': False,
 'SAVE_OUTPUTS': True,
 'TARGET_NAME': 'GALLSTONES, Cholelithiasis',
 'additional_target_cui_terms_list': ['C0008325', 'C0008311'],
 'OUTPUT_RES_PREFIX': 'gallstone_'}

#### Get negative candidates
* feat imp
* Negative interesting + make fae explanation
* maybe check that model didn't find as interesting ? 

In [8]:
import sys
# sys.path.append(r'D:\Research2\MedRAG')
# sys.path.append(r'D:\Research2\MedRAG\src')
sys.path.append('/mnt/d/Research2/MedRAG/')
sys.path.append('/mnt/d/Research2/MedRAG/src')

sys.path.append('/Users/oferd2/Library/CloudStorage/OneDrive-MedtronicPLC/Documents/research/MedRag/')
sys.path.append('/Users/oferd2/Library/CloudStorage/OneDrive-MedtronicPLC/Documents/research/MedRag/src')
sys.path.append('/mnt/d/Research2/MedRAG/')
## change to path with medrag and it's downloaded corpus!
# os.chdir('/mnt/d/Research2/MedRAG/')
# os.chdir('/Users/oferd2/Library/CloudStorage/OneDrive-MedtronicPLC/Documents/research/MedRag')
from src.medrag import MedRAG
from tqdm import tqdm
import json

from util import get_predictions_from_medrag

# CORPUS_DIR_CACHE_PATH = '/Users/oferd2/Library/CloudStorage/OneDrive-MedtronicPLC/Documents/research/MedRag/corpus'
CORPUS_DIR_CACHE_PATH = '/mnt/d/Research2/MedRAG/corpus'

In [9]:
medrag = MedRAG(llm_name="OpenScholar/Llama-3.1_OpenScholar-8B", rag=True,
                # retriever_name="BM25" # "BM25" #"RRF-2"#"MedCPT" - RRF2 woudl take ~ 11 hours for pubmed
                retriever_name="RRF-2" ## fast enough with small data
# , corpus_name= "MedCorp" # big
                , corpus_name="MedText"
                ,HNSW=True ,corpus_cache=True
                ,db_dir= CORPUS_DIR_CACHE_PATH#'/Users/oferd2/Library/CloudStorage/OneDrive-MedtronicPLC/Documents/research/MedRag/corpus',
               )

No sentence-transformers model found with name ncbi/MedCPT-Query-Encoder. Creating a new one with CLS pooling.
No sentence-transformers model found with name ncbi/MedCPT-Query-Encoder. Creating a new one with CLS pooling.
Initializing the document extracter...
Initialization finished!


In [10]:
%%time
# all_results = []
if generate_basic_neg_candidates:
    for config in all_configs:
        print("--"*50,"\n", config["TARGET_NAME"],"\n","--"*30)
        
        ### pre-extracted positives:
        df_pos_candidates = pd.read_csv(config['CANDIDATE_NOVEL_CUIS_FILEPATH']) # cases filtered for novelty
        print(f"df_pos_candidates # rows {df_pos_candidates.shape[0]}, nunq raw_name: {df_pos_candidates['raw_name'].nunique()}")
        # df_hits = df_hits.loc[~df_hits["raw_name"].isin(df_pos_candidates["raw_name"])]
        
        df_hits = link_kg_concepts_slim(FEATURES_REPORT_PATH=config["FEATURES_REPORT_PATH"], CANDIDATE_NOVEL_CUIS_FILEPATH=config["CANDIDATE_NOVEL_CUIS_FILEPATH"],
                     TARGET_NAME=config["TARGET_NAME"],additional_target_cui_terms_list=config["additional_target_cui_terms_list"],
                        MIN_EVIDENCE_FILTER=5 if FastRun else 2,
                                   FAST=FastRun, SAVE_OUTPUTS = False,get_simplePathLengths=False)
        print("\n","--"*20)
        print(df_hits.shape[0],'df_hits, before filtering out pos')
        df_hits= df_hits.loc[~df_hits["raw_name"].isin(df_pos_candidates["raw_name"])]
        # print(df_hits.shape[0],'df_hits, after filtering out pos')
        df_hits= df_hits.loc[~df_hits["feature_name"].isin(df_pos_candidates["feature_name"])] # drops a few more
        print(df_hits.shape[0],'df_hits, after filtering out pos')
        
        icu_feature_terms = pd.read_csv(config["FEATURES_REPORT_PATH"])
        # icu_feature_terms = icu_feature_terms.loc[(icu_feature_terms["feature_importance"] > 0) |(icu_feature_terms["MutualInfoTarget"] >= 0.01)|(icu_feature_terms["p_val"] <= 0.05) ]  # filter a bit - optionally
        print(icu_feature_terms["name"].nunique(),"# unique names before cleaning")
        icu_feature_terms = wrangle_df_icu(icu_feature_terms,filter_feat_imp=False)
        print(icu_feature_terms["name"].nunique(),"# unique names post-cleaning") # considerable drop after cleaning up feature names, missing values, etc'
        
        ## note: Not using raw/cleaned so there may be disparity! can sanity check by looking at "no features list " 
        ## coverage of features with any linked KG entity
        num_inputFeat = icu_feature_terms["name"].nunique()
        num_covered_feat = df_hits["feature_name"].nunique()
        num_features_not_covered = num_inputFeat- num_covered_feat
        print("num_inputFeat:",num_inputFeat,"num_covered_feat:",num_covered_feat,"num_features_not_covered:",num_features_not_covered)
        
        ## direct link, ignores 1-2 hops:
        print((df_hits.groupby("feature_name")["KG_Hits"].max()>0).agg(["mean","sum","size","count"]).round(2))
        ## % cases/features with a link with targets in KG
        mean_kg_known = (100*(df_hits.groupby("feature_name")["KG_Hits"].max()>0).mean()).round(2)
        print("mean_kg_known:",mean_kg_known)
        
        df_sub = df_hits.loc[(df_hits["shortest_path_length"]<=2) & (df_hits["KG_Hits"]>0)].drop_duplicates(["raw_name","feature_name","cui"])
        print(f"df_sub ({df_sub.shape[0]} cases, with direct kg hits)")
        print(df_sub.select_dtypes("O").nunique())
    
        df_sub = df_sub.sort_values(["shortest_path_length","p_val","KG_Hits"],ascending=False)
        ## lemmatize/drop some case sensitives - redundnat a bit ,done later on the queries also
        df_sub["base_word"] = df_sub["feature_name"].apply(lemma.lemmatize).str.lower().str.strip()
        df_sub = df_sub.drop_duplicates("base_word").drop(columns=["base_word"]).reset_index(drop=True)
        fname_result_df = mini_pubmed_search(df_sub)
        print("--"*50,"\n","pubmed searched","\n","--"*30)
        # display(fname_result_df)
        print(fname_result_df.shape[0])
    
        fname_result_df = fname_result_df.loc[fname_result_df["Co-occurrence Count"]>11]
        print(fname_result_df.shape[0])
        fname_result_df = fname_result_df.drop_duplicates("Query")
        print(fname_result_df.shape[0])
    
        fname_result_df = fname_result_df.merge(df_sub,left_on=["Query"],right_on=["feature_name"])
    
        ## clean results for less semantic redundnacy, e.g. same queries ieven if feature is different: 
        fname_result_df["base_word"] = fname_result_df["Query"].apply(lemma.lemmatize).str.lower().str.strip()
    
        fname_result_df.drop_duplicates(subset=["base_word"],inplace=True)
        ## drop some trivial cases, like subdisease/history in target, i.e family history of Cholelithiasis), or "stones" as query (for gallstones")
        s1 = fname_result_df.shape[0]
        fname_result_df = fname_result_df[~fname_result_df.apply(lambda row: row["base_word"].lower() in row["Target"].lower(), axis=1)]
        print(s1 - fname_result_df.shape[0],"# rows where feature in target name dropped")
    
        fname_result_df = fname_result_df.round(3).sort_values(["shortest_path_length","Cooccurrence P-Value",
                                     # "Co-occurrence Count",
                                     "p_val"],ascending=False).drop(columns=["base_word"])
        print("\n","--"*25)
        fname_result_df = fname_result_df[['feature_name', 'raw_name', 'shortest_path_length','Co-occurrence Count','Cooccurrence P-Value',
                                            'Odds Ratio', 
                                           "Query Count",
           'feature_importance', 'p_val', 'corr', 'MutualInfoTarget', 'cui',"Target"]]
        ## Filter to remove cases with obviously low cooccurrence/ i.e appering together less than expected 
        print(fname_result_df.shape[0])
        fname_result_df = fname_result_df.loc[fname_result_df['Cooccurrence P-Value']>0.34].reset_index(drop=True)
        print(fname_result_df.shape[0],"without rarely coocc cases")
        print(config["OUTPUT_RES_PREFIX"])
    
        ## Do deduplication
        df = fname_result_df.copy()
        if df.shape[0]>1:
            print(df.shape[0])
            clean_feats = deduplicate_texts(df["feature_name"], use_difflib=True, string_cutoff=0.95, distance_threshold=0.4)
            df = df.loc[df["feature_name"].isin(clean_feats)]
            if df.shape[0]>1:
                clean_feats = deduplicate_texts(df["raw_name"], use_difflib=False, distance_threshold=0.25)
                df = df.loc[df["raw_name"].isin(clean_feats)]
            print(df.shape[0],"After semantic clustering of feats")
        df = df.head(90) # we only need some
        fname_result_df = df

        print("medrag boring LLM")
        boring_prompts = generate_boring_prompts(df)
        boring_pred,boring_cot = get_predictions_from_medrag(medrag, boring_prompts, 'boring_prompt', 'boring_options',K=32)
        df["boring_pred"] = boring_pred
        df["boring_cot"] = boring_cot
        df["boring"] = df["boring_pred"]=="A"
    
        display(df)
        if Save_Outputs:
            df.to_csv(os.path.join(OutputPath, config["OUTPUT_RES_PREFIX"]+"_negs.csv"),index=False)
else:
    df = pd.read_csv(os.path.join(OutputPath, config["OUTPUT_RES_PREFIX"]+"_negs.csv"))
    clean_feats = deduplicate_texts(df["feature_name"], use_difflib=True, string_cutoff=0.95, distance_threshold=0.5)
    df = df.loc[df["feature_name"].isin(clean_feats)]

    fname_result_df = df.copy()


---------------------------------------------------------------------------------------------------- 
 GALLSTONES, Cholelithiasis 
 ------------------------------------------------------------
df_pos_candidates # rows 1381, nunq raw_name: 697
After filtering KG min count
12915192
TARGET_NAME (For Entity-KG linking GALLSTONES, Cholelithiasis
GALLSTONES C0008350
Cholelithiasis C0008350
list_target_cuis ['C0008350', 'C0008325', 'C0008311', 'C0242216']
(1447, 11) # feats preclean
(1442, 11) # feats after mini filt and load
153:
 No Entity feats: ['Qualifications College or University degree', 'Job code at visit managers', 'hypersensitivity anaphylaxis', 'Frequency of friend/family visits About once a week', 'Job code at visit management', 'medication glucosamine product', 'joint', 'Job code at visit', 'Basophill count', 'Job code at visit professionals', 'medication text', 'medication glucosamine', 'crohns', 'Caffeine drink within last hour', 'Job code at visit fitters', 'Stiffness method'

Calculating Paths: 100%|████████████████████████████████████████████████████████████████████████| 779/779 [00:00<00:00, 29972.78it/s]


(779, 3) df_path_lengths
(1677, 11) df_hits

 ----------------------------------------
1677 df_hits, before filtering out pos
285 df_hits, after filtering out pos
1447 # unique names before cleaning
(1447, 11) # feats preclean
854 # unique names post-cleaning
num_inputFeat: 854 num_covered_feat: 133 num_features_not_covered: 721
mean       0.59
sum       79.00
size     133.00
count    133.00
Name: KG_Hits, dtype: float64
mean_kg_known: 59.4
df_sub (110 cases, with direct kg hits)
feature_name        78
cui                 53
cui_nomenclature    53
cui_def             53
raw_name            78
dtype: int64
Targets: ['(Cholelithiasis) OR (Gallstone) OR (Gallbladder disease) OR (cholecystitis) OR (Cholangitis)']
76 Rows, 76 # unique Feature Names


Targets: ['(Cholelithiasis) OR (Gallstone) OR (Gallbladder disease) OR (cholecystitis) OR (Cholangitis)']


70
70
---------------------------------------------------------------------------------------------------- 
 pubmed searched 
 ------------------------------------------------------------
70
59
59
3 # rows where feature in target name dropped

 --------------------------------------------------
56
47 without rarely coocc cases
gallstone_
47
47 # inputs
# Applying string-level merging (difflib) first...
47 # Deduplicated.
47 # inputs
47 # Deduplicated.
47 After semantic clustering of feats
medrag boring LLM


...

Targets: ['(Depression) OR (Depressive disorder)']


218
218
---------------------------------------------------------------------------------------------------- 
 pubmed searched 
 ------------------------------------------------------------
218
192
192
1 # rows where feature in target name dropped

 --------------------------------------------------
191
122 without rarely coocc cases
depression_
122
122 # inputs
# Applying string-level merging (difflib) first...
111 # Deduplicated.
111 # inputs
111 # Deduplicated.
111 After semantic clustering of feats
medrag boring LLM


Processing rows: 100%|███████████████████████████████████████████████████████████████████████████████| 90/90 [09:30<00:00,  6.34s/it]


,feature_name,raw_name,shortest_path_length,Co-occurrence Count,Cooccurrence P-Value,Odds Ratio,Query Count,feature_importance,p_val,corr,MutualInfoTarget,cui,Target,boring_pred,boring_cot,boring
0,Ethnic background British,Ethnic background_British,2,220,1.0,2.889,4389,0.006,0.122,0.001,0.063,C0015031,(Depression) OR (Depressive disorder),B,The association between 'Ethnic background Bri...,False
1,Ethnic background Mixed,Ethnic background_Mixed,2,581,1.0,4.323,7941,0.000,0.029,0.001,0.000,C0015031,(Depression) OR (Depressive disorder),B,The association between 'Ethnic background Mix...,False
2,Ethnic background,Ethnic background_None,2,11307,1.0,3.165,209110,0.000,0.010,-0.001,0.000,C0015031,(Depression) OR (Depressive disorder),B,The relationship between ethnic background and...,False
3,Doctor diagnosed COPD (chronic obstructive pul...,Doctor diagnosed COPD (chronic obstructive pul...,2,96,1.0,2.130,2563,0.000,0.006,-0.001,0.000,C0600260,(Depression) OR (Depressive disorder),A,The relationship between chronic obstructive p...,True
4,Illnesses of mother 3 None (group 1),Illnesses of mother_3_None (group 1),2,12,1.0,9.803,79,0.000,0.005,-0.001,0.000,C0687744,(Depression) OR (Depressive disorder),A,The feature 'Illnesses of mother_3_None (group...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,irritable bowel,"Non-cancer illness code, self-reported | Array...",1,2141,1.0,6.823,19363,0.000,0.000,-0.000,0.000,C0022104,(Depression) OR (Depressive disorder),A,The association between irritable bowel and de...,True
96,medication amitriptyline,Treatment/medication code | Array 0__amitripty...,1,900,1.0,22.782,3065,0.004,0.000,-0.002,0.002,C0002600,(Depression) OR (Depressive disorder),A,Amitriptyline is a tricyclic antidepressant th...,True
97,medication citalopram,Treatment/medication code | Array 0__citalopram,1,2218,1.0,74.356,3856,0.018,0.000,-0.004,0.003,C0008845,(Depression) OR (Depressive disorder),A,The association between the feature'medication...,True
98,medication fluoxetine,Treatment/medication code | Array 0__fluoxetine,1,3141,1.0,63.756,5850,0.002,0.000,-0.000,0.002,C0016365,(Depression) OR (Depressive disorder),A,The association between the feature'medication...,True


CPU times: user 40min 29s, sys: 4min 53s, total: 45min 23s
Wall time: 46min 14s


In [11]:
fname_result_df.loc[fname_result_df["Odds Ratio"]<1]

,feature_name,raw_name,shortest_path_length,Co-occurrence Count,Cooccurrence P-Value,Odds Ratio,Query Count,feature_importance,p_val,corr,MutualInfoTarget,cui,Target,boring_pred,boring_cot,boring
25,"Poisoning by analgesics, antipyretics, and ant...","missing_Poisoning by analgesics, antipyretics,...",2,21,0.514,0.977,1198,0.003,0.0,0.001,0.014,C0032343,(Depression) OR (Depressive disorder),A,The association between 'Poisoning by analgesi...,True


In [12]:
fname_result_df.loc[fname_result_df["Odds Ratio"]>1.1]

,feature_name,raw_name,shortest_path_length,Co-occurrence Count,Cooccurrence P-Value,Odds Ratio,Query Count,feature_importance,p_val,corr,MutualInfoTarget,cui,Target,boring_pred,boring_cot,boring
0,Ethnic background British,Ethnic background_British,2,220,1.0,2.889,4389,0.006,0.122,0.001,0.063,C0015031,(Depression) OR (Depressive disorder),B,The association between 'Ethnic background Bri...,False
1,Ethnic background Mixed,Ethnic background_Mixed,2,581,1.0,4.323,7941,0.000,0.029,0.001,0.000,C0015031,(Depression) OR (Depressive disorder),B,The association between 'Ethnic background Mix...,False
2,Ethnic background,Ethnic background_None,2,11307,1.0,3.165,209110,0.000,0.010,-0.001,0.000,C0015031,(Depression) OR (Depressive disorder),B,The relationship between ethnic background and...,False
3,Doctor diagnosed COPD (chronic obstructive pul...,Doctor diagnosed COPD (chronic obstructive pul...,2,96,1.0,2.130,2563,0.000,0.006,-0.001,0.000,C0600260,(Depression) OR (Depressive disorder),A,The relationship between chronic obstructive p...,True
4,Illnesses of mother 3 None (group 1),Illnesses of mother_3_None (group 1),2,12,1.0,9.803,79,0.000,0.005,-0.001,0.000,C0687744,(Depression) OR (Depressive disorder),A,The feature 'Illnesses of mother_3_None (group...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,irritable bowel,"Non-cancer illness code, self-reported | Array...",1,2141,1.0,6.823,19363,0.000,0.000,-0.000,0.000,C0022104,(Depression) OR (Depressive disorder),A,The association between irritable bowel and de...,True
96,medication amitriptyline,Treatment/medication code | Array 0__amitripty...,1,900,1.0,22.782,3065,0.004,0.000,-0.002,0.002,C0002600,(Depression) OR (Depressive disorder),A,Amitriptyline is a tricyclic antidepressant th...,True
97,medication citalopram,Treatment/medication code | Array 0__citalopram,1,2218,1.0,74.356,3856,0.018,0.000,-0.004,0.003,C0008845,(Depression) OR (Depressive disorder),A,The association between the feature'medication...,True
98,medication fluoxetine,Treatment/medication code | Array 0__fluoxetine,1,3141,1.0,63.756,5850,0.002,0.000,-0.000,0.002,C0016365,(Depression) OR (Depressive disorder),A,The association between the feature'medication...,True


In [13]:
fname_result_df.columns

Index(['feature_name', 'raw_name', 'shortest_path_length',
       'Co-occurrence Count', 'Cooccurrence P-Value', 'Odds Ratio',
       'Query Count', 'feature_importance', 'p_val', 'corr',
       'MutualInfoTarget', 'cui', 'Target', 'boring_pred', 'boring_cot',
       'boring'],
      dtype='object')

In [14]:
len(boring_prompts)

90

In [15]:
boring_prompts

[{'feature': 'Ethnic background British',
  'raw_feature': 'Ethnic background_British',
  'target': '(Depression) OR (Depressive disorder)',
  'boring_prompt': "Evaluate the feature 'Ethnic background_British' in relation to predicting the target disease: '(Depression) OR (Depressive disorder)'. The feature has a positive correlation with the target disease). \n Is the association between the feature 'Ethnic background British' (raw: 'Ethnic background-British') and 'Depression OR Depressive disorder', already known, boring, obvious (trivial, e.g. is a known risk / protective factor) or is part of known risk factors (or related pathways or mechanism)? ",
  'boring_options': {'A': 'Yes, it is well known or explained by existing knowledge.',
   'B': 'No, it is not well known or explained by existing knowledge.'}},
 {'feature': 'Ethnic background Mixed',
  'raw_feature': 'Ethnic background_Mixed',
  'target': '(Depression) OR (Depressive disorder)',
  'boring_prompt': "Evaluate the featur

In [16]:
# LL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct" # works, less stable than 3.0 model?
# LL_NAME ="meta-llama/Llama-3.2-3B-Instruct"
# LL_NAME ="meta-llama/Llama-3.2-1B-Instruct"

# LL_NAME ="allenai/Llama-3.1-Tulu-3-8B" # loads ok

LL_NAME = "OpenScholar/Llama-3.1_OpenScholar-8B"  # openscholar moidel # error loading? worked now.. 
## could do open scholar rag - https://github.com/AkariAsai/OpenScholar
# LL_NAME ="OpenAI/gpt-4o-mini"
# LL_NAME ="OpenAI/gpt-4o" # hits token limits on tier 1 account. tokens per min (TPM): Limit 30000, Used 24125, Requested 7561. https://platform.openai.com/account/rate-limits

# if Fast_Run:
    # LL_NAME ="meta-llama/Llama-3.2-1B-Instruct"
    # LL_NAME ="meta-llama/Llama-3.2-3B-Instruct"

LL_NAME

'OpenScholar/Llama-3.1_OpenScholar-8B'

In [17]:
# %%time
# medrag = MedRAG(llm_name="OpenScholar/Llama-3.1_OpenScholar-8B", rag=True,
#                 # retriever_name="BM25" # "BM25" #"RRF-2"#"MedCPT" - RRF2 woudl take ~ 11 hours for pubmed
#                 retriever_name="RRF-2" ## fast enough with small data
# # , corpus_name= "MedCorp" # big
#                 , corpus_name="MedText"
#                 ,HNSW=True ,corpus_cache=True
#                 ,db_dir= CORPUS_DIR_CACHE_PATH#'/Users/oferd2/Library/CloudStorage/OneDrive-MedtronicPLC/Documents/research/MedRag/corpus',
#                )

# boring_pred,boring_cot = get_predictions_from_medrag(medrag, boring_prompts, 'boring_prompt', 'boring_options',K=30)
# df["boring_pred"] = boring_pred
# df["boring_cot"] = boring_cot
# df["boring"] = df["boring_pred"]=="A"

No sentence-transformers model found with name ncbi/MedCPT-Query-Encoder. Creating a new one with CLS pooling.
No sentence-transformers model found with name ncbi/MedCPT-Query-Encoder. Creating a new one with CLS pooling.
Initializing the document extracter...
Initialization finished!


ValueError: Could not load model OpenScholar/Llama-3.1_OpenScholar-8B with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/transformers/pipelines/base.py", line 289, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/transformers/modeling_utils.py", line 4174, in from_pretrained
    hf_quanti

In [19]:
boring_pred

['B',
 'B',
 'B',
 'A',
 'A',
 'B',
 'A',
 'A',
 'A',
 'A',
 'B',
 'A',
 'A',
 'B',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'B',
 'A',
 'A',
 'B',
 'B',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A']

In [20]:
boring_cot

["The association between 'Ethnic background British' and 'Depression OR Depressive disorder' is not explicitly mentioned in the provided documents. However, it is known that cultural and ethnic factors can influence the expression and experience of mental health conditions, including depression. For instance, Document [28] mentions that Asian patients, including those from the UK, may deny depressed mood and present with more somatic complaints, and may be at higher risk of extrapyramidal side effects of psychotropic medications. This suggests that cultural and ethnic factors can impact the presentation and treatment of depression. However, the specific association between 'Ethnic background British' and 'Depression OR Depressive disorder' is not explicitly stated in the provided documents.",
 "The association between 'Ethnic background Mixed' and 'Depression OR Depressive disorder' is not explicitly mentioned in the provided documents. However, the documents do discuss various risk f